# Decoding Quantum Hypergraph Product Codes

In [1]:
import numpy as np
from tqdm import tqdm
import qecstruct as qc
import qecsim.paulitools as pt

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.ticker import FormatStrFormatter

from mdopt.mps.utils import marginalise, create_custom_product_state
from mdopt.contractor.contractor import mps_mpo_contract
from mdopt.optimiser.utils import (
    SWAP,
    COPY_LEFT,
    XOR_BULK,
    XOR_LEFT,
    XOR_RIGHT,
)
from examples.decoding.decoding import (
    apply_constraints,
    apply_bitflip_bias,
)
from examples.decoding.decoding import (
    decode_css,
    pauli_to_mps,
    css_code_checks,
    css_code_logicals,
    css_code_logicals_sites,
    css_code_constraint_sites,
    generate_pauli_error_string,
)

In [2]:
NUM_BITS = 4
NUM_EXPERIMENTS = 50

SEED = 123
seed_seq = np.random.SeedSequence(SEED)

max_bond_dims = [128, 64, 32, 16, 8]
error_rates = np.linspace(1e-2, 0.2, 11)
failures_statistics = {}

for CHI_MAX in max_bond_dims:
    print(f"CHI_MAX = {CHI_MAX}")
    for ERROR_RATE in tqdm(error_rates):
        failures = []

        for l in range(NUM_EXPERIMENTS):
            new_seed = seed_seq.spawn(1)[0]
            rng = np.random.default_rng(new_seed)
            random_integer = rng.integers(1, 10**8 + 1)
            SEED = random_integer

            CHECK_DEGREE, BIT_DEGREE = 4, 3
            NUM_CHECKS = int(BIT_DEGREE * NUM_BITS / CHECK_DEGREE)
            if NUM_BITS / NUM_CHECKS != CHECK_DEGREE / BIT_DEGREE:
                raise ValueError("The Tanner graph of the code must be bipartite.")
            regular_code = qc.random_regular_code(
                NUM_BITS, NUM_CHECKS, BIT_DEGREE, CHECK_DEGREE, qc.Rng(SEED)
            )
            hgp_code = qc.hypergraph_product(regular_code, regular_code)

            error = generate_pauli_error_string(len(hgp_code), ERROR_RATE, seed=SEED)
            error = pauli_to_mps(error)

            _, success = decode_css(
                code=hgp_code,
                error=error,
                chi_max=CHI_MAX,
                bias_type="Depolarizing",
                bias_prob=ERROR_RATE,
                renormalise=True,
                silent=True,
                contraction_strategy="Optimised",
            )

            failures.append(1 - success)

        failures_statistics[NUM_BITS, CHI_MAX, ERROR_RATE] = failures

CHI_MAX = 128


  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
failure_rates = {}
error_bars = {}

for CHI_MAX in max_bond_dims:
    for ERROR_RATE in error_rates:
        failure_rates[NUM_BITS, CHI_MAX, ERROR_RATE] = np.mean(
            failures_statistics[NUM_BITS, CHI_MAX, ERROR_RATE]
        )
        error_bars[NUM_BITS, CHI_MAX, ERROR_RATE] = (
            np.std(failures_statistics[NUM_BITS, CHI_MAX, ERROR_RATE])
            * 1.96  # Standart error of the mean with 95% confidence interval
            / NUM_EXPERIMENTS
        )

In [ ]:
plt.figure(figsize=(5, 4))

green_cmap = matplotlib.colormaps["viridis_r"]
norm = Normalize(vmin=0, vmax=len(max_bond_dims) - 1)

for index, CHI_MAX in enumerate(max_bond_dims):
    plt.errorbar(
        error_rates,
        [failure_rates[NUM_BITS, CHI_MAX, ERROR_RATE] for ERROR_RATE in error_rates],
        yerr=[error_bars[NUM_BITS, CHI_MAX, ERROR_RATE] for ERROR_RATE in error_rates],
        fmt="o--",
        label=f"System size: {NUM_BITS}, max bond dim: {CHI_MAX}",
        linewidth=3,
        color=green_cmap(norm(index)),
    )

plt.legend(fontsize=7)
plt.xlabel("Error rate")
plt.ylabel("Failure rate")
plt.grid()

plt.show()